In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import shutil
import mysql.connector as msql
from mysql.connector import Error
from geopy.geocoders import Nominatim 
geolocator = Nominatim(user_agent="MakroAnalyse")
import psycopg2
from psycopg2 import Error
import psycopg2.extras  

c:\Users\leand\miniconda3\envs\datascience\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [20]:
print('Introduzca la dirección donde se encuentren los conjuntos de datos: ')

direccion = input()
carpeta_datos = direccion + '\\'
lista_datos = []
for file in os.listdir(carpeta_datos):
    if file.endswith('.csv'):
        lista_datos.append(file)
    if file.endswith('.json'):
        lista_datos.append(file)
lista_datos = sorted(lista_datos)

Introduzca la dirección donde se encuentren los conjuntos de datos: 


In [21]:
ruta1 = (carpeta_datos + 'product_category_name_translation.csv')
ruta2 = (carpeta_datos + 'olist_closed_deals_dataset.csv')
ruta3 = (carpeta_datos + 'geolocation_dataset_brazil.csv')
ruta4 = (carpeta_datos + 'olist_products_dataset.csv')
ruta5 = (carpeta_datos + 'olist_order_payments_dataset.csv')
ruta6 = (carpeta_datos +  'olist_orders_dataset.csv')
ruta7 = (carpeta_datos + 'olist_order_reviews_dataset.csv')
ruta8 = (carpeta_datos + 'olist_order_items_dataset.csv')
ruta9 = (carpeta_datos + 'olist_sellers_dataset.csv')
ruta10 = (carpeta_datos + 'olist_customers_dataset.csv')
rutaestados = (carpeta_datos + 'estados.json')

In [22]:
def etl_categoryproduct(ruta1):

    data_category_product = pd.read_csv(ruta1)
    data_category_product.loc[71] = ['sin_categoria','NoCategory']
    data_category_product.reset_index(inplace=True)
    data_category_product.rename(columns={'index':'product_name_id'}, inplace=True)

    data_category_product.to_csv('data_normalizada\category_product.csv', index=False)

    return data_category_product

In [23]:
def etl_closed(ruta2):
    
    data_closed = pd.read_csv(ruta2)
    data_closed['won_date'] = pd.to_datetime(data_closed['won_date'])
    data_closed.drop(columns=['declared_monthly_revenue', 'sr_id', 'sdr_id', 'lead_behaviour_profile', \
        'has_company','lead_type','has_gtin','business_type' ,'average_stock', 'declared_product_catalog_size'], inplace=True)
    data_closed.reset_index(inplace=True)
    data_closed.rename(columns={'index':'closed_id'}, inplace=True)
    data_closed.dropna(inplace=True)

    data_closed.to_csv('data_normalizada\closed_deal.csv', index=False)

In [24]:
def etl_geolocalizacion(ruta3, rutaestados):
    geo = pd.read_csv(ruta3)
    geo.drop_duplicates(inplace=True)
    geo = geo.rename(columns={'geolocation_zip_code_prefix': 'zip_code', 'geolocation_lat':'geolatitud', \
        'geolocation_lng':'geolongitud', 'geolocation_city':'city', 'geolocation_state' : 'state'})
    dfj = pd.read_json(rutaestados)
    
    geo = geo.merge(dfj[['nome', 'sigla']], right_on='sigla', left_on='state')

    geo.rename(columns={'nome':'state_name'}, inplace=True)

    geo['city'] = geo['city'].str.title()

    geo = geo.drop_duplicates('zip_code')

    geo = geo.reset_index(drop=True)

    geo = geo[['zip_code', 'geolatitud', 'geolongitud', 'city', 'state_name']]


    geo.loc[geo.geolongitud > 0, 'geolongitud'] = geo.geolongitud * (-1)
    geo.loc[geo.geolatitud > 5, 'geolatitud'] = geo.geolatitud * (-1)


    a = (geo.geolongitud < -73) | (geo.geolongitud > -34) | (geo.geolatitud < -33)

    geo['full_data'] = geo.state_name + ',' + geo.city 

    geo.loc[a, 'gcode'] = geo[a]['full_data'].apply(geolocator.geocode)
    geo.loc[a,'lat'] = [g.latitude for g in geo[a]['gcode']]
    geo.loc[a,'long'] = [g.longitude for g in geo[a]['gcode']]
    
    geo.loc[a, 'geolatitud'] = geo[a]['lat']
    geo.loc[a, 'geolongitud'] = geo[a]['long']
    
    geo.drop(columns= ['gcode', 'lat', 'long', 'full_data'], inplace=True)



    geo['zip_code'] = geo['zip_code'].astype(str) 

    geo.to_csv('data_normalizada\geolocation.csv', index=False)

    return geo

In [25]:
def etl_product(ruta4):
    data_products = pd.read_csv(ruta4)
    data_products.drop(columns=['product_name_lenght', 'product_description_lenght'], axis=1, inplace=True)
    ## UTILIZAR FUNCION DE ETL_CATEGORYPRODUCT
    datamerge = merge1
    data_products.drop(columns=['product_photos_qty'], inplace=True)
    data_products['product_category_name'] = data_products['product_category_name'].fillna('sin_categoria')
    data_products = pd.merge(datamerge, data_products)
    data_products.drop(columns='product_category_name', inplace=True)
    data_products.drop(columns='product_name_id', inplace= True)

    #Utilizamos la media para rellenar espacios nulos
    product_weight_median = data_products['product_weight_g'].median()
    product_length_median = data_products['product_length_cm'].median()
    product_height_median = data_products['product_height_cm'].median()
    product_width_median = data_products['product_width_cm'].median()
    data_products['product_weight_g'].fillna(product_weight_median, inplace=True)
    data_products['product_length_cm'].fillna(product_length_median, inplace=True)
    data_products['product_height_cm'].fillna(product_height_median, inplace=True)
    data_products['product_width_cm'].fillna(product_width_median, inplace=True)   
    data_products = data_products.reindex(columns = ['product_id', 'product_category_name_english', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'])

    data_products.to_csv('data_normalizada\product.csv', index=False)

In [26]:
def etl_payment(ruta5):
    data_payment = pd.read_csv(ruta5)
    data_payment['payment_type'] = data_payment['payment_type'].replace('boleto', 'voucher')

    data_payment.reset_index(inplace=True)
    data_payment.rename(columns={'index':'payment_id'}, inplace=True)
    data_payment['payment_id'] = data_payment['payment_id'].astype(str)

    data_payment.to_csv('data_normalizada\payment.csv', index=False)

In [27]:
def etl_order(ruta6):
    data_order = pd.read_csv(ruta6)
    date_cols = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']

    for j in date_cols:
        data_order[j] = pd.to_datetime(data_order[j])
    data_order.drop([47552], inplace= True)
    diferencia_compra_aprobacion = data_order['order_approved_at'] - data_order['order_purchase_timestamp']
    diferencia_aprobacion_envio = data_order['order_delivered_carrier_date'] - data_order['order_approved_at']

    Q1 = diferencia_compra_aprobacion.dt.days.quantile(0.25) 
    Q3 = diferencia_compra_aprobacion.dt.days.quantile(0.75)
    IQR = Q3 - Q1
    BI = Q1 - 1.5*IQR
    BS = Q3 + 1.5*IQR
    out = (diferencia_compra_aprobacion.dt.days<BI) | (diferencia_compra_aprobacion.dt.days>BS)

    promedio_demora_aprobacion = diferencia_compra_aprobacion[~out].mean()

    data_order.loc[diferencia_aprobacion_envio.dt.days < 0, 'order_approved_at'] = data_order.order_purchase_timestamp + pd.Timedelta(promedio_demora_aprobacion) 

    diferencia_aprobacion_envio_2 = data_order['order_delivered_carrier_date'] - data_order['order_approved_at']

    Q1 = diferencia_aprobacion_envio_2.dt.days.quantile(0.25) 
    Q3 = diferencia_aprobacion_envio_2.dt.days.quantile(0.75)
    IQR = Q3 - Q1
    BI = Q1 - 1.5*IQR
    BS = Q3 + 1.5*IQR
    out = (diferencia_aprobacion_envio_2.dt.days<BI) | (diferencia_aprobacion_envio_2.dt.days>BS)

    promedio_demora_salida = diferencia_aprobacion_envio_2[~out].mean()

    data_order.loc[diferencia_aprobacion_envio_2.dt.days < 0, 'order_delivered_carrier_date'] = data_order.order_approved_at + pd.Timedelta(promedio_demora_salida) 

    diferencia_salida_llegada = data_order['order_delivered_customer_date'] - data_order['order_delivered_carrier_date']

    Q1 = diferencia_salida_llegada.dt.days.quantile(0.25) 
    Q3 = diferencia_salida_llegada.dt.days.quantile(0.75)
    IQR = Q3 - Q1
    BI = Q1 - 1.5*IQR
    BS = Q3 + 1.5*IQR
    out = (diferencia_salida_llegada.dt.days<BI) | (diferencia_salida_llegada.dt.days>BS)

    promedio_demora_llegada = diferencia_salida_llegada[~out].mean()

    data_order.loc[diferencia_salida_llegada.dt.days < 0, 'order_delivered_customer_date'] = data_order.order_delivered_carrier_date + pd.Timedelta(promedio_demora_llegada) 
    
    data_order.loc[(data_order.order_status == 'delivered') & (data_order.order_approved_at.isnull()), 'order_approved_at'] = data_order.order_purchase_timestamp + pd.Timedelta(promedio_demora_aprobacion) 

    data_order.loc[(data_order.order_status == 'delivered') & (data_order.order_delivered_carrier_date.isnull()), 'order_delivered_carrier_date'] = data_order.order_approved_at + pd.Timedelta(promedio_demora_salida) 

    data_order.loc[(data_order.order_status == 'delivered') & (data_order.order_delivered_customer_date.isnull()), 'order_delivered_customer_date'] = data_order.order_delivered_carrier_date + pd.Timedelta(promedio_demora_llegada) 

    lista = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

    for i in lista:
        data_order[i] = pd.to_datetime(data_order[i]).round('s')

    data_order.replace({np.nan: None}, inplace= True)
 
    data_order.to_csv('data_normalizada\data_order.csv', index=False)

In [28]:
def etl_review(ruta7):
    data_order_review = pd.read_csv(ruta7)
    data_order_review['review_creation_date'] = pd.to_datetime(data_order_review['review_creation_date'])
    data_order_review['review_id'] = data_order_review.index
    data_order_review['review_id'] = data_order_review['review_id'].astype(str)
    data_order_review.drop(columns=['review_comment_title', 'review_comment_message', 'review_answer_timestamp'], inplace=True)

    data_order_review.to_csv('data_normalizada\data_review.csv', index=False)

In [29]:
def etl_items(ruta8):
    data_order= pd.read_csv(ruta8)
    data_order['shipping_limit_date'] = pd.to_datetime(data_order['shipping_limit_date'])
    data_order['percentagePF'] = ( data_order['freight_value'] * 100 ) / data_order['price']
    data_order.rename(columns= {'order_item_id' : 'quantity'}, inplace= True)
    data_order = data_order.reindex(columns = ['order_id', 'product_id', 'seller_id', 'shipping_limit_date', 'quantity', 'price', 'freight_value', 'percentagePF'])
    data_order.reset_index(inplace=True)
    data_order.rename(columns={'index':'orderitem_id'}, inplace=True)

    data_order.to_csv('data_normalizada\item.csv', index=False)

In [30]:
def etl_seller(ruta9):
    
    df = pd.read_csv(ruta9)
    df.rename(columns = {'seller_zip_code_prefix': 'zip_code', 'seller_city': 'city', 'seller_state': 'state'}, inplace= True)
    df['city'] = df['city'].str.title()
    df['zip_code'] = df['zip_code'].astype(str)
    dfgeo = merge_geo
    df = df[['seller_id', 'zip_code']].merge(dfgeo[['zip_code', 'city', 'state_name']], on='zip_code')

    df.to_csv('data_normalizada\seller.csv', index=False)

In [31]:
def etl_customer(ruta10):
    
    df = pd.read_csv(ruta10)
    dfgeo = merge_geo
    df.rename(columns = {'customer_zip_code_prefix': 'zip_code', 'customer_city': 'city', 'customer_state': 'state'}, inplace= True)
    df['city'] = df['city'].str.title()
    df.rename(columns={'customer_zip_code_prefix': 'zip_code'}, inplace=True)
    df['zip_code'] = df['zip_code'].astype(str)
    df = df[['customer_id', 'customer_unique_id', 'zip_code']].merge(dfgeo[['zip_code', 'city', 'state_name']], on='zip_code')
    
    df.to_csv('data_normalizada\customer.csv', index=False)


In [32]:
funciones = [etl_categoryproduct, etl_closed, etl_geolocalizacion, etl_product, etl_payment, etl_order, etl_review, etl_items, etl_seller, etl_customer]
rutas = [ruta1, ruta2, ruta3, ruta4, ruta5, ruta6, ruta7, ruta8, ruta9, ruta10]
j = 0
p = 0

for i in funciones:
   print('Normalizando el archivo', rutas[j].split(carpeta_datos)[1])
   
   if i == etl_geolocalizacion:
      merge_geo = i(rutas[j], rutaestados)
   elif i == etl_categoryproduct:
      merge1 = i(rutas[j])
   else:
      i(rutas[j])

   j = j + 1
   p = p + 1
   
print('El proceso de normalización ha finalizado exitosamente')

Normalizando el archivo product_category_name_translation.csv
Normalizando el archivo olist_closed_deals_dataset.csv
Normalizando el archivo geolocation_dataset_brazil.csv
Normalizando el archivo olist_products_dataset.csv
Normalizando el archivo olist_order_payments_dataset.csv
Normalizando el archivo olist_orders_dataset.csv
Normalizando el archivo olist_order_reviews_dataset.csv
Normalizando el archivo olist_order_items_dataset.csv
Normalizando el archivo olist_sellers_dataset.csv
Normalizando el archivo olist_customers_dataset.csv
El proceso de normalización ha finalizado exitosamente


In [1]:
carpeta_data_norm = r'C:\Users\leand\Desktop\Data Science (Henry)\Proyectos\Proyecto_grupal\DS-Proyecto_Grupal_Olist\data_normalizada'
lista_archivos = []
for file in os.listdir(carpeta_data_norm):
    if file.endswith('.csv'):
        lista_archivos.append(file)

lista_archivos

['category_product.csv',
 'closed_deal.csv',
 'customer.csv',
 'data_order.csv',
 'data_review.csv',
 'geolocation.csv',
 'item.csv',
 'payment.csv',
 'product.csv',
 'seller.csv']

In [3]:
lista_archivos[1] = 'geolocation.csv'
lista_archivos[2] = 'customer.csv'
lista_archivos[3] = 'seller.csv'
lista_archivos[4] = 'data_order.csv'
lista_archivos[5] = 'product.csv'
lista_archivos[6] = 'item.csv'
lista_archivos[7] = 'payment.csv'
lista_archivos[8] = 'data_review.csv'
lista_archivos[9] = 'closed_deal.csv'

In [3]:
for archivo in lista_archivos:
    my_file = open(carpeta_data_norm + '\\' + archivo)

    try:
        conn_string = "host=%s dbname=%s user=%s password=%s" % ('localhost', 'db_olist', 'postgres', '2685')
        conn = psycopg2.connect(conn_string)
        cursor = conn.cursor()
        SQL_STATEMENT = """COPY {} FROM stdin DELIMITER ',' CSV header;""".format(archivo[:-4])
        cursor.copy_expert(sql=SQL_STATEMENT, file= my_file)
        print('Carga subida a la DB')
        conn.commit()
    except (Exception, psycopg2.Error) as error:
        print(error)

Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB
Carga subida a la DB


DELTA ORDER

In [35]:
try: 
    conn_string = "host=%s dbname=%s user=%s password=%s" % ('localhost', 'db_olist', 'postgres', '2685')
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE table_aux (order_id varchar, customer_id varchar, order_status varchar, order_purchase_timestamp timestamp,\
    order_approved_at timestamp, order_delivered_carrier_date timestamp, order_delivered_customer_date timestamp,\
    order_estimated_delivery_date timestamp, PRIMARY KEY (order_id));")

    conn.commit()

except (Exception, psycopg2.Error) as error:
    print(error)

ACÁ HAY QUE CREAR UNA FUNCIÓN DELTA QUE SEA IGUAL A ORDER, PERO RECIBA COMO PARÁMETRO LA RUTA DEL DELTA Y LUEGO LO EXPORTE A UN CSV order_delta

In [36]:
try:
    #TRAIGO EL ARCHIVO DELTA NORMALIZADO
    my_file = open(r"C:\Users\leand\Desktop\Data Science (Henry)\Proyectos\Proyecto_grupal\DS-Proyecto_Grupal_Olist\data_normalizada\data_order.csv")

    conn_string = "host=%s dbname=%s user=%s password=%s" % ('localhost', 'db_olist', 'postgres', '2685')
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()

    SQL_STATEMENT = "COPY table_aux FROM stdin DELIMITER ',' CSV header;"
    cursor.copy_expert(sql=SQL_STATEMENT, file= my_file)
    

    conn.commit()

except (Exception, psycopg2.Error) as error:
    print(error)

In [37]:
try:
    # INSERTAMOS LOS CUSTOMER_ID QUE SEAN NUEVOS EN LA TABLA CUSTOMER PARA NO PERDER LA INTEGRIDAD
    
    conn_string = "host=%s dbname=%s user=%s password=%s" % ('localhost', 'db_olist', 'postgres', '2685')
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()


    cursor.execute("INSERT INTO customer (customer_id)\
    SELECT customer_id FROM (SELECT * FROM table_aux\
    WHERE table_aux.customer_id NOT IN (SELECT customer_id FROM customer)) as pepito;")
    conn.commit()

except (Exception, psycopg2.Error) as error:
    print(error)

In [11]:
try:
    # INSERTAMOS LOS AUXILIARES EN ORDENES DONDE NO SE REPITA ORDER_ID
    
    conn_string = "host=%s dbname=%s user=%s password=%s" % ('localhost', 'db_olist', 'postgres', '2685')
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()


    cursor.execute("INSERT INTO data_order (order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date)\
    SELECT order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date FROM (SELECT * FROM table_aux\
    WHERE table_aux.order_id NOT IN (SELECT order_id FROM data_order)) as pepito;")


    conn.commit()

except (Exception, psycopg2.Error) as error:
    print(error)

inserción o actualización en la tabla «data_order» viola la llave foránea «data_order_fk_customer»
DETAIL:  La llave (customer_id)=(9ef432eb6251297304e76186b10a928d) no está presente en la tabla «customer».

